
!pip install dash pandas plotly

In [ ]:
# System-level package installation
import sys
import subprocess

def install_packages():
    packages = ['dash', 'pandas', 'plotly']
    for package in packages:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
    print("All packages installed successfully!")
    print("Please restart the kernel and run all cells")

install_packages()

All packages installed successfully!
Please restart the kernel and run all cells


In [ ]:
# Import required libraries with error handling
try:
    import pandas as pd
    import plotly.express as px
    from dash import Dash, dcc, html
    from dash.dependencies import Input, Output
    print("All imports successful! Proceeding with dashboard setup...")
except ImportError as e:
    print(f"Error: {e}")
    print("Please make sure you've restarted the kernel after package installation")
    raise

All imports successful! Proceeding with dashboard setup...


In [9]:
# Load and prepare data
df = pd.read_csv(r"F:\Git\Internship\dataset\clean_retail_data.csv")

# Process data
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Hour'] = df['InvoiceDate'].dt.hour
df['Month'] = df['InvoiceDate'].dt.month
df['Year'] = df['InvoiceDate'].dt.year
df['Sales'] = df['Quantity'] * df['UnitPrice']

# Calculate key metrics
total_sales = df['Sales'].sum()
total_customers = df['CustomerID'].nunique()
total_products = df['Description'].nunique()

print("Data processed successfully!")

Data processed successfully!


In [12]:
# Initialize dashboard with Dash
app = Dash(__name__)

# Layout with Bootstrap CSS
app.layout = html.Div([
    # Title
    html.H1("Retail Analytics Dashboard", style={'textAlign': 'center', 'color': '#2c3e50', 'marginBottom': 30}),
    
    # Filters row
    html.Div([
        html.Div([
            html.Label("Select Country:"),
            dcc.Dropdown(
                id='country-filter',
                options=[{'label': x, 'value': x} for x in sorted(df['Country'].unique())],
                value='United Kingdom',
                clearable=False
            )
        ], style={'width': '30%', 'display': 'inline-block'}),
        
        html.Div([
            html.Label("Select Time Period:"),
            dcc.DatePickerRange(
                id='date-range',
                start_date=df['InvoiceDate'].min(),
                end_date=df['InvoiceDate'].max()
            )
        ], style={'width': '40%', 'display': 'inline-block', 'marginLeft': '5%'})
    ], style={'marginBottom': 30}),
    
    # KPI Cards
    html.Div([
        html.Div([
            html.H4(id='total-sales'),
            html.P("Total Sales")
        ], className='kpi-card'),
        html.Div([
            html.H4(id='avg-order'),
            html.P("Average Order Value")
        ], className='kpi-card'),
        html.Div([
            html.H4(id='total-customers'),
            html.P("Total Customers")
        ], className='kpi-card')
    ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginBottom': 30}),
    
    # Charts
    html.Div([
        dcc.Graph(id='sales-trend'),
        dcc.Graph(id='top-products')
    ], style={'display': 'flex', 'flexWrap': 'wrap'}),
    
    html.Div([
        dcc.Graph(id='hourly-sales'),
        dcc.Graph(id='country-sales')
    ], style={'display': 'flex', 'flexWrap': 'wrap'})
])

# Callbacks
@app.callback(
    [Output('sales-trend', 'figure'),
     Output('top-products', 'figure'),
     Output('hourly-sales', 'figure'),
     Output('country-sales', 'figure'),
     Output('total-sales', 'children'),
     Output('avg-order', 'children'),
     Output('total-customers', 'children')],
    [Input('country-filter', 'value'),
     Input('date-range', 'start_date'),
     Input('date-range', 'end_date')]
)
def update_dashboard(country, start_date, end_date):
    # Filter data
    mask = (df['Country'] == country) & \
           (df['InvoiceDate'] >= start_date) & \
           (df['InvoiceDate'] <= end_date)
    filtered_df = df[mask]
    
    # Sales Trend
    sales_trend = filtered_df.resample('M', on='InvoiceDate')['Sales'].sum().reset_index()
    fig1 = px.line(sales_trend, x='InvoiceDate', y='Sales', 
                   title='Monthly Sales Trend')
    
    # Top Products
    top_products = filtered_df.groupby('Description')['Sales'].sum().nlargest(10)
    fig2 = px.bar(top_products, title='Top 10 Products by Sales',
                  orientation='h')
    
    # Hourly Sales Pattern
    hourly_sales = filtered_df.groupby('Hour')['Sales'].mean()
    fig3 = px.line(hourly_sales, title='Average Sales by Hour')
    
    # Sales by Country
    country_sales = filtered_df.groupby('Country')['Sales'].sum()
    fig4 = px.bar(country_sales, title='Sales by Country')
    
    # KPI Values
    total_sales = f"£{filtered_df['Sales'].sum():,.0f}"
    avg_order = f"£{filtered_df.groupby('InvoiceNo')['Sales'].sum().mean():,.2f}"
    total_customers = f"{filtered_df['CustomerID'].nunique():,}"
    
    return fig1, fig2, fig3, fig4, total_sales, avg_order, total_customers

if __name__ == '__main__':
    print("Starting dashboard server at http://127.0.0.1:8050/")
    app.run(host='127.0.0.1', port=8050, debug=True)

Starting dashboard server at http://127.0.0.1:8050/
